# Classification Analysis
This notebook contains work done for classification analysis on the Mozambique
dataset from IPUMS.

## Load Dependencies

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    roc_auc_score,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (
    Dataset,
    DataLoader
)
from torchvision import transforms, models

from src.utils.ipums_extract import load_ipums_from_pkl

## Load Data

In [2]:
PKL_PATH = Path(r"data/mozambique.pkl")
mig1_df, mig5_df = load_ipums_from_pkl(PKL_PATH)

print(mig1_df.shape)
print(mig5_df.shape)

(5929529, 66)
(4974569, 66)


## Compute Class Distribution

In the cells below, we examine the class distribution for our two response
variables. From these results, we can see that we have an extremely unbalanced
dataset. Therefore, for the models implemented in Scikit-Learn we will use
class weights to automatically balance the classes by assigning lower weights
to majority class samples and higher weights to minority class samples. For the
neural network, we will employ dropout layers and a weighted loss function to in
principle accomplish the same goal. The next cell outlines papers where this is
discussed.

"A systematic study of the class imbalance problem in convolutional neural
networks" by Buda et al. (2018)

"Focal Loss for Dense Object Detection" by Lin et al. (2017)

"Learning from Imbalanced Data" by He et al. (2009)

"Dropout: A Simple Way to Prevent Neural Networks from Overfitting" by
Srivastava et al. (2014)

"Deep Learning for imbalanced multimedia data classification" by Pouyanfar et
al. (2018)

"Class-Balanced Loss Based on Effective Number of Samples" by Cui et al. (2019)

In [28]:
print(mig1_df['MIGRATE1'].value_counts())

MIGRATE1
0    5860462
1      69067
Name: count, dtype: int64


In [29]:
print(mig5_df['MIGRATE5'].value_counts())

MIGRATE5
0    4746396
1     228173
Name: count, dtype: int64


## Create Development Splits (Train/Val/Test)

In [18]:
# Split Hyperparameters
SEED = 5523
VAL_RATIO = 0.15
TEST_RATIO = 0.15
TRAIN_RATIO = 1 - VAL_RATIO - TEST_RATIO
assert np.sum([TRAIN_RATIO, VAL_RATIO, TEST_RATIO]) == 1

In [ ]:
# MIG1
X1 = mig1_df.drop(columns=['MIGRATE1'], inplace=False, axis=1)
y1 = mig1_df['MIGRATE1'].values

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1,
                                                        test_size=TEST_RATIO,
                                                        random_state=SEED,
                                                        stratify=y1)

X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train,
                                                      test_size=VAL_RATIO,
                                                      random_state=SEED,
                                                      stratify=y1_train)

In [27]:
# MIG5
X5 = mig5_df.drop(columns=['MIGRATE5'], inplace=False, axis=1)
y5 = mig5_df['MIGRATE5'].values

X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5,
                                                        test_size=TEST_RATIO,
                                                        random_state=SEED,
                                                        stratify=y5)

X5_train, X5_val, y5_train, y5_val = train_test_split(X5_train, y5_train,
                                                      test_size=VAL_RATIO,
                                                      random_state=SEED,
                                                      stratify=y5_train)

## Random Forest
In this section, we define a random forest using Scikit-Learn to be applied for
classifying census samples' migration status.

In [34]:
def train_random_forest(X_train, y_train, X_val, y_val):
    """Train Random Forest classifier with class weights for handling the
    imbalanced dataset and with hyperparameter optimization."""

    # Define Parameter Grid
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [5, 10],
        'min_samples_leaf': [2, 4],
        'max_features': ['sqrt'],
        'class_weight': ['balanced'],
        'bootstrap': [True],
        'random_state': [SEED]
    }

    # Define Base Model
    forest = RandomForestClassifier()

    # Perform GridSearchCV
    grid_search = GridSearchCV(
        estimator=forest,
        param_grid=param_grid,
        cv=3,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=True,
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Print Best Found Hyperparameters
    print("Best Hyperparameters:")
    for param, val in grid_search.best_params_.items():
        print(f"{param}: {val}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")

    # Evaluate Using Validation Set
    y_val_pred = best_model.predict(X_val)
    y_val_prob = best_model.predict_proba(X_val)[:, 1]

    metrics = {
        'accuracy': accuracy_score(y_val, y_val_pred),
        'precision': precision_score(y_val, y_val_pred, average='weighted'),
        'recall': recall_score(y_val, y_val_pred, average='weighted'),
        'f1': f1_score(y_val, y_val_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_val, y_val_prob)
    }

    # Print Validation Set Performance
    print("\nValidation Set Performance:")
    for metric, val in metrics.items():
        print(f"{metric}: {val:.4f}")
    
    # Print Feature Importance Analysis
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print("T\nop 10 Most Important Features")
        print(feature_importance.head(10))
    
    return best_model, metrics

In [38]:
def evaluate_random_forest(model, X_test, y_test):
    """Evaluate trained Random Forest on test set."""

    # Compute Predictions
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1]

    # Compute Metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred, average='weighted'),
        'recall': recall_score(y_test, y_test_pred, average='weighted'),
        'f1': f1_score(y_test, y_test_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_test, y_test_prob)
    }

    return metrics

In [ ]:
# Train and Evaluate for MIG1
forest1, forest1_results = train_random_forest(
    X1_train, y1_train, X1_val, y1_val
)

forest1_test_results = evaluate_random_forest(forest1, X1_test, y1_test)
print(f"\n{forest1_test_results}")

In [ ]:
# Train and Evaluate for MIG5
forest5, forest5_results = train_random_forest(
    X5_train, y5_train, X5_val, y5_val
)

forest5_test_results = evaluate_random_forest(forest5, X5_test, y5_test)
print(f"\n{forest5_test_results}")

## Support Vector Machine (SVM)
In this section, we define a support vector machine model using Scikit-Learn to
be applied for classifying census samples' migration status.

In [41]:
def train_svc(X_train, y_train, X_val, y_val):
    """Train Support Vector classifier with class weights for handling the
    imbalanced dataset and with hyperparameter optimization."""

    # Define Parameter Grid
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale'],
        'class_weight': ['balanced'],
        'random_state': [SEED]
    }

    # Define Base Model
    svc = SVC(probability=True, max_iter=1000)

    # Perform GridSearchCV
    grid_search = GridSearchCV(
        estimator=svc,
        param_grid=param_grid,
        cv=3,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=True,
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Print Best Found Hyperparameters
    print("Best Hyperparameters:")
    for param, val in grid_search.best_params_.items():
        print(f"{param}: {val}")
    print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")

    # Evaluate Using Validation Set
    y_val_pred = best_model.predict(X_val)
    y_val_prob = best_model.predict_proba(X_val)[:, 1]

    metrics = {
        'accuracy': accuracy_score(y_val, y_val_pred),
        'precision': precision_score(y_val, y_val_pred, average='weighted'),
        'recall': recall_score(y_val, y_val_pred, average='weighted'),
        'f1': f1_score(y_val, y_val_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_val, y_val_prob)
    }

    # Print Validation Set Performance
    print("\nValidation Set Performance:")
    for metric, val in metrics.items():
        print(f"{metric}: {val:.4f}")
    
    # Print Feature Importance Analysis
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print("T\nop 10 Most Important Features")
        print(feature_importance.head(10))
    
    return best_model, metrics

In [39]:
def evaluate_svc(model, X_test, y_test):
    """Evaluate trained Random Forest on test set."""

    # Compute Predictions
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1]

    # Compute Metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred, average='weighted'),
        'recall': recall_score(y_test, y_test_pred, average='weighted'),
        'f1': f1_score(y_test, y_test_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_test, y_test_prob)
    }

    return metrics

In [ ]:
# Train and Evaluate for MIG1
svc1, svc1_results = train_svc(
    X1_train, y1_train, X1_val, y1_val
)

svc1_test_results = evaluate_svc(svc1, X1_test, y1_test)
print(f"\n{svc1_test_results}")

In [ ]:
# Train and Evaluate for MIG5
svc5, svc5_results = train_svc(
    X5_train, y5_train, X5_val, y5_val
)

svc5_test_results = evaluate_svc(svc5, X5_test, y5_test)
print(f"\n{svc5_test_results}")

## Neural Network
In this section, we define a artificial neural network using PyTorch to be
applied for classifying census samples' migration status.

In [42]:
class NeuralNetwork(nn.Module):
    """
    Neural Network with imbalanced data handling.
     * Dropout (default=0.3) prevents overfitting the majority class
     * Batch normalization stabilizes training with imbalanced batches
    """
    def __init__(self, input_dim, hidden_dims=[256, 128, 64],
                 num_classes=2, dropout=0.3):
        super(NeuralNetwork, self).__init__()

        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, num_classes))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [43]:
def compute_class_weights(y):
    """Compute balanced class weights for loss function."""
    classes = np.unique(y)
    class_counts = np.bincount(y)
    weights = len(y) / (len(classes) * class_counts)
    return torch.FloatTensor(weights)

In [45]:
class CustomDataset(Dataset):
    """Custom Dataset wrapper for migration data."""
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X.values)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [46]:
def train_neural_net(X_train, y_train, X_val, y_val, n_epochs=50,
                     batch_size=256, learning_rate=0.001):
    """Train neural network with class-weighted loss."""

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Compute class weights
    class_weights = compute_class_weights(y_train)

    # Create datasets and loaders
    train_dataset = CustomDataset(X_train, y_train)
    val_dataset = CustomDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize neural net
    model = NeuralNetwork(
        input_dim=X_train.shape[1],
        hidden_dims=[256, 128, 64],
        dropout=0.3
    ).to(device)

    # Define loss function, optimizer, learning rate schedule
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # Training Loop
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 10
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_f1': []
    }

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_x)

            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        
        # Validation performance
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                outputs = model(batch_x)

                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        # Compute metrics
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_f1 = f1_score(all_labels, all_preds, average='weighted')

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f},"
              f" Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Add early stopping (prevent overfitting of majority class)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    return model, history

In [47]:
def evaluate_neural_network(model, X_test, y_test):
    """Evaluate trained neural network."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # Test dataset and loader
    test_dataset = CustomDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    # Prediction storage
    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(batch_y.numpy())
    
    # Compute metrics
    metrics = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted'),
        'recall': recall_score(all_labels, all_preds, average='weighted'),
        'f1': f1_score(all_labels, all_preds, average='weighted'),
        'roc_auc': roc_auc_score(all_labels, all_preds)
    }

    return metrics

In [ ]:
# Train and Evaluate for MIG1
nn1, nn1_results = train_neural_net(
    X1_train, y1_train, X1_val, y1_val,
    n_epochs=50,
    batch_size=256,
    learning_rate=0.001
)

nn1_test_results = evaluate_neural_network(nn1, X1_test, y1_test)
print(f"\n{nn1_test_results}")

In [ ]:
# Train and Evaluate for MIG5
nn5, nn5_results = train_neural_net(
    X5_train, y5_train, X5_val, y5_val,
    n_epochs=50,
    batch_size=256,
    learning_rate=0.001
)

nn5_test_results = evaluate_neural_network(nn5, X5_test, y5_test)
print(f"\n{nn5_test_results}")